In [12]:
from openpyxl import Workbook
import pandas as pd
import re
import xlwings 

In [13]:
dat = {
    'cod_alumno':'cod',
    'des_doc_identidad':'doc_type',
    'did_alumno':'doc_num',
    'ape_paterno':'paternal_name',
    'ape_materno':'maternal_name',
    'nom_alumno':'name',
    'tel_alu_movil':'cel_num',
    'tel_alumno':'phone_num',
    'coe_alumno':'inst_email',
    'coe_alu_personal':'per_email',
    'anio_ingreso':'in_year',
    'periodo_egreso':'grad_per',
    'escuela':'school'
}
col = list(dat.values())
archivos =['./EGRESADOS/BASE DE DATOS AUDITORIA EGRESADOS.xlsx','./EGRESADOS/BASE DE DATOS CONTABILIDAD EGRESADOS.xlsx','./EGRESADOS/BASE DE DATOS EGRESADOS GESTIÓN PÚBLICA.xlsx']

In [14]:
data = [pd.read_excel(archivo,sheet_name=1,usecols=dat.keys(),dtype="string") for archivo in archivos]

In [15]:
def document(val):
    if len(val)!=8 and len(val)!=10:
        val = None
    return val

In [16]:
db = pd.concat(data).rename(columns=dat).drop_duplicates().reset_index(drop=True)
db = db[db.doc_num.notnull()]
db.doc_num = db.doc_num.apply(document)
db = db[db.doc_num.notnull()]
db.fillna('sin_registro',inplace=True)

In [17]:
for val in db.doc_num:
    if len(val)==9:
        print(val)

In [18]:
def phone(val):
    if len(val)<7:
        val = 'sin_registro'
    if val[3]=='-':
        val=val[:3]+val[4:]
    if re.search('^0',val):
        val=val[2:]
    if not re.search('^\d+',val) or re.search('^0',val):
        val='sin_registro'
    if len(val)!=7 and val != 'sin_registro':
        val = 'sin_registro'
    return val

def movil(val):
    if len(val)<9:
        val = 'sin_registro'
    if val[5]=='-':
        val = val[0:5]+val[6:]
    if not re.search('^\d+',val):
        val = 'sin registro'
    if len(val)>9 and re.search('^51',val):
        val = val[2:]
    if len(val)>9:
        val = val[0:9]
    if not re.search('^9',val):
        val ='sin registro'
    if len(val)!=9:
        val = 'sin_registro'
    return val
    

def income(val):
    if len(val)!=4:
        val = 'sin_registro'
    return val

def graduated(val):
    if not re.search('^\d+',val) or re.search('^0',val):
        val = 'sin_registro'
    if val!='sin_registro' and val[4]!='-':
        val = val[:4] + '-' + val[4]
    return val

In [19]:
db.phone_num=db.phone_num.apply(phone)
db.cel_num=db.cel_num.apply(movil)
db.in_year=db.in_year.apply(income)
db.grad_per=db.grad_per.apply(graduated)

In [20]:
db = db[col]
db.columns = db.columns.str.upper()
db = db.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})
db.to_excel('EGRESA2.xlsx', sheet_name = 'EGRESADOS', index=False)
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('EGRESA2.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()

In [21]:
unchange = ['COD','DOC_TYPE','DOC_NUM','PATERNAL_NAME','MATERNAL_NAME','NAME','SCHOOL']
change = ['PHONE_NUM','CEL_NUM','INST_EMAIL','PER_EMAIL','IN_YEAR','GRAD_PER']

In [22]:
student = pd.read_excel('./EGRESA2.xlsx',usecols=unchange,dtype='string')
temporal = pd.read_excel('./EGRESA2.xlsx',usecols=change,dtype='string')

In [23]:
student = student.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})
student.to_excel('STUDENT.xlsx', sheet_name = 'STUDENT', index=False)
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('STUDENT.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()

In [24]:
temporal = temporal.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})
temporal.to_excel('TEMPORAL.xlsx', sheet_name = 'TEMPORAL', index=False)
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('TEMPORAL.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()